In [1]:
from json import loads
import os.path, time
import csv
import re
import shutil
import itertools
import traceback
from glob import glob
#from openpyxl import load_workbook
# import pandas as pd

▼ 파싱할 json 경로를 입력하세요. (ex: E:\1_Check\SLF_001\84\RESULT\MALICIOUS\)

In [2]:
def jsonFile():
       
    return glob(dirName + '*.json')

In [3]:
def extract_categories(report_items, category_count):
    """
    report_items 리스트를 순회하며 category 정보를 수집하는 재귀 함수.
    """
    for item in report_items:
        category = item.get("category")
        if category:
            category_count[category] = category_count.get(category, 0) + 1
        
        # 하위 reportItems가 있는 경우 재귀 호출
        sub_items = item.get("reportItems")
        if sub_items and isinstance(sub_items, list):
            extract_categories(sub_items, category_count)

In [4]:
def cdrParsing(report):
    
    with open(report, 'r', encoding='utf-8') as json_file:
        json_data = loads(json_file.read())
    
        cdrRoot = json_data.get("fileAnalysisResults",None)[0]
        balthaTime = cdrRoot.get("elapsedTimes",None).get("BALTHAZAR_ANALYSIS",None).get("endTime",None) - cdrRoot.get("elapsedTimes",None).get("BALTHAZAR_ANALYSIS",None).get("startTime",None)
        totalTime = json_data.get("allElapsedTime",None)
        
        targetData = {
            "targetId": cdrRoot.get("targetId",None),
            "sha256": cdrRoot.get("sha256",None),
            "fileName": cdrRoot.get("fileName",None),
            #"fileFormat": cdrRoot.get("fileFormat",None),
            "mimeType": cdrRoot.get("mimeType",None),
            "cdrResult": cdrRoot.get("cdrInfo",None).get("status",None),
            "filesize": cdrRoot.get("size",None)/1024/1024,
            "balthaTime": balthaTime,
            "allElapsedTime": totalTime,
        }
        
        if cdrRoot.get("cdrInfo",None).get("status",None) != "NOT_RUN": 
        
            sanitizeData = {
                "cdrResult": cdrRoot.get("cdrInfo",None).get("status",None),
                "cdrStatus": cdrRoot.get("cdrResult",None).get("cdrMarsReport",None).get("statusInfo",None).get("message",None),
                "cdrElapsedTime": cdrRoot.get("cdrResult",None).get("cdrMarsReport",None).get("statusInfo",None).get("elapsedTime",None),
                "cdrDetailMessage": cdrRoot.get("cdrResult",None).get("cdrMarsReport",None).get("statusInfo",None).get("detailMessage",None),
                "cdrDetectedFileType": cdrRoot.get("cdrResult",None).get("detectedFileType",None),
            }

            itemCategory = cdrRoot.get("cdrResult",None).get("cdrMarsReport",None).get("reportItems",None)
            #print(itemCategory)
            itemCount = len(itemCategory)

            sanitizeItem = {}

            for num in range(itemCount):
                category = itemCategory[num].get("category")
                categoryTotal = itemCategory[num].get("total")

                # 해당 카테고리가 이미 존재하면 +1, 아니면 1로 초기화
                if category in sanitizeItem:
                    sanitizeItem[category] += categoryTotal
                else:
                    sanitizeItem[category] = categoryTotal

            combineData = {**targetData, **sanitizeData, **sanitizeItem}
            #print(combineData)
            
        else:
            combineData = {**targetData}

        return combineData

In [5]:
dataField =  {
    "targetId":"", "sha256": "", "fileName":"", "filesize":"", "mimeType":"", "cdrDetectedFileType":"", "cdrResult":"", "cdrStatus":"", 
    "balthaTime":"", "cdrElapsedTime":"", "allElapsedTime":"", "cdrDetailMessage":"", 
    "ACTIVEX":"", "ATTACHED_TEMPLATE":"", "DDE":"", "FORMULA":"", "FRAME_SET":"", "HYPERLINK":"", "LINKED_OBJECT":"", "MACRO":"", 
    "EXCEL4":"", "OLE":"", "SCRIPT":"", "POST_SCRIPT":"", "EMBEDDED_FILE":"", "INVALID_ENTRY":"", "ORPHAN_ENTRY":"", "UNUSED_ENTRY":"", "ATTACHMENT":"", 
    "HTML":"", "IMAGE":"", "MEDIA":"", "QR_CODE":"", "META_DATA":"", "COMMENT":"", "UNSPECIFIED_DATA":"", "LNK_FIELD":"", "SUSPICIOUS_DATA":"", 
    "INVALID_CHARACTER":"", "REG_EXP":"", "DRAWING_ML":"", "XFA": ""
}

In [9]:
dirName = input("json 경로를 입력하세요.")
resultDir = input("결과 취합된 CSV 파일의 저장위치를 입력하세요.")
resultFileName = input("CSV 파일 이름을 입력하세요.")
resultCSV = resultDir + resultFileName

json 경로를 입력하세요.\\192.168.2.11\slf_003\Report\
결과 취합된 CSV 파일의 저장위치를 입력하세요.\\192.168.2.11\slf_003\Report\
CSV 파일 이름을 입력하세요.csv_test.csv


In [10]:
#jsonReport = jsonFile()

with open(resultCSV, "w", encoding="utf-8-sig", newline="") as csv_file:
    
    writer = csv.DictWriter(csv_file, fieldnames=dataField.keys())
    writer.writeheader()

    for report in glob(dirName + "*.json"):
        
        if report.find(".zip.json") != -1:
            pass
        
        else:
            data = cdrParsing(report)
            writer.writerow(data)

csv_file.close()